In [1]:
import os
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=True,
                       min_detection_confidence = 0.9)

In [4]:
data_dir = "Data"
data = []
labels = []

for i in sorted(os.listdir(data_dir)):
    if i == ".DS_Store":
        pass
    else:
        for j in os.listdir(os.path.join(data_dir,i)):
            data_aux = []
            img = cv2.imread(os.path.join(data_dir,i,j))
            img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        img_rgb,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style()
                    )
                plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
                plt.show()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data'

In [5]:
X_train,X_test,y_train,y_test = train_test_split(
    np.array(data),labels,
    test_size=0.15,random_state=22,
    shuffle=True
)

## train

model = RandomForestClassifier(random_state=22)
model.fit(X_train,y_train)

pred = model.predict(X_test)

score= accuracy_score(y_test,pred)
print(score)

ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
## save the model 

f = open("model.pkl","wb")
pickle.dump(model,f)
f.close()

In [ ]:
model_dict = pickle.load(open("model.pkl","rb"))
model = model_dict['model']

In [ ]:
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8,
                    min_tracking_confidence=0.8) as hands:
    while cap.isOpened():
        data_aux = []
        X_ = []
        y_ = []

        ret,frame = cap.read()
        H,W , _ =  frame.shape

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_rgb = cv2.flip(frame_rgb,1)
        frame_rgb.flags.writeable = False
        results = hands.process(frame_rgb)
        frame_rgb.flags.writeable = True
        frame_rgb = cv2.cvtColor(frame_rgb,cv2.COLOR_BGR2RGB)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame_rgb,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(28,255,2),thickness=5,
                                           circle_radius = 10),
                )
#
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    X=hand_landmarks.landmark[i].X
                    y=hand_landmarks.landmark[i].y
                    data_aux.append(X)
                    data_aux.append(y)
                    X_.append(X)
                    y_.append(y)

            x1 = int(min(X_)* W)-10
            y1 = int(min(y_)* H)-10

            x2 = int(max(X_)* W)-10
            y2 = int(max(y_)* H)-10

            prediction = model.predict([np.array(data_aux)[0:42]])[0]
            cv2.rectangle(frame_rgb,(x1,y1-10),(x2,y2),(255,99,173),6)
            cv2.putText(frame_rgb,prediction,(x1,y1),cv2.FONT_HERSHEY_DUPLEX,5(255,0,0),5,cv2.LINE_AA)
            cv2.imshow("frame",frame_rgb)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

cap.release()
cv2.destroyAllWindows()

